In [1]:
import numpy as np
import collections
import re
from pymatgen.io.vasp import inputs
import datetime

Parse

In [2]:
with open('bestsqs_0819.out') as f:
    data = f.readlines()
    
cell_vec_raw = data[:3]
cell_vec= []
for i in cell_vec_raw:
    cell_vec.append([float(j) for j in i.split()])
cell_vec = np.array(cell_vec)

lat_vec_raw = data[3:6]
lat_vec= []
for i in lat_vec_raw:
    lat_vec.append([float(j) for j in i.split()])
lat_vec = np.array(lat_vec)

atoms_raw = data[6:]
atoms = {}
c = 0
for i in atoms_raw:
    string = i.replace('\n','')
    atoms[f'{str(string.split()[3])}{c}'] = [float(string.split()[0]), float(string.split()[1]), float(string.split()[2])]
    c += 1
atoms_sorted = dict(collections.OrderedDict(sorted(atoms.items())))

Calc

In [3]:
decart_basis = []
for i in [0,1,2]:
    x = lat_vec[i][0]*cell_vec[i][0] + lat_vec[i][1]*cell_vec[1][0] + lat_vec[i][2]*cell_vec[2][0]
    y = lat_vec[i][0]*cell_vec[i][1] + lat_vec[i][1]*cell_vec[1][1] + lat_vec[i][2]*cell_vec[2][1]
    z = lat_vec[i][0]*cell_vec[i][2] + lat_vec[i][1]*cell_vec[1][2] + lat_vec[i][2]*cell_vec[2][2]
    decart_basis.append([x,y,z])
decart_basis

[[-1.590245, -2.754385, -7.09096],
 [-1.590245, 2.754385, 0.0],
 [6.36098, -11.01754, -7.09096]]

In [4]:
new_coords_atoms_srted = {}
dArrLatVecInv = np.linalg.inv(decart_basis)
for i in atoms_sorted:
    x = atoms_sorted[i][0]
    y = atoms_sorted[i][1]
    z = atoms_sorted[i][2]
    x_ = x * cell_vec[0][0] + y * cell_vec[1][0] + z * cell_vec[2][0]
    y_ = x * cell_vec[0][1] + y * cell_vec[1][1] + z * cell_vec[2][1]
    z_ = x * cell_vec[0][2] + y * cell_vec[1][2] + z * cell_vec[2][2]
    x__ = x_ * dArrLatVecInv[0][0] + y_ * dArrLatVecInv[1][0] + z_ * dArrLatVecInv[2][0]
    y__ = x_ * dArrLatVecInv[0][1] + y_ * dArrLatVecInv[1][1] + z_ * dArrLatVecInv[2][1]
    z__ = x_ * dArrLatVecInv[0][2] + y_ * dArrLatVecInv[1][2] + z_ * dArrLatVecInv[2][2]
    new_coords_atoms_srted[i] = [x__, y__, z__]

In [5]:
translations_vector_matrix = decart_basis #lat_vec*cell_vec

atoms_final_coords = []
symbols = []

for i in new_coords_atoms_srted:
    symbol = re.sub('\d', '', i)
    symbols.append(symbol)
    c = ''
    for j in new_coords_atoms_srted[i]:
        c += str(j) + ' '
    atoms_final_coords.append(c + symbol)
atoms_final_coords_as_string = '\n'.join(atoms_final_coords)

In [6]:
unique_symbols = collections.Counter(map(tuple, symbols))
elems, counts = '', ''
for i in unique_symbols:
    elems += ''.join(i) + ' '
    counts += str(unique_symbols[i]) + ' '

In [7]:
v = ''
for i in translations_vector_matrix:
    for j in i:
        v += str(j) + " "
    v += '\n'

In [8]:
POSCAR = f'None\n1.0\n{v}{elems}\n{counts}\nDirect\n{atoms_final_coords_as_string}'

Check

In [9]:
read_poscar = inputs.Poscar.from_string(POSCAR)
struct = read_poscar.structure
comp_dict = struct.composition.fractional_composition.as_dict()
cTa = comp_dict['Ta']/(comp_dict['Ta']+comp_dict['Zr']) * 100
cZr = comp_dict['Zr']/(comp_dict['Ta']+comp_dict['Zr']) * 100

current_date = datetime.datetime.now()
filename = f"{cTa}TaB2_{current_date.strftime('%y%m%d_%H%M%S')}.vasp"

comment = f"# mcsqs generated structure of solid solution {cTa}%-TaB2 + {cZr}%-ZrB2 on date {current_date.strftime('%y%m%d_%H%M%S')}"

print(f'TaB2: {cTa} %')
print(f'Natoms: {sum(read_poscar.natoms)}')
print(struct.composition.as_dict())

TaB2: 80.0 %
Natoms: 90
defaultdict(<class 'float'>, {'B': 60.0, 'Ta': 24.0, 'Zr': 6.0})


Save

In [10]:
POSCAR = f'{comment}\n1.0\n{v}{elems}\n{counts}\nDirect\n{atoms_final_coords_as_string}'

In [11]:
with open(filename, 'w') as f:
    f.write(POSCAR)
print(f'poscar saved as {filename}')

poscar saved as 80.0TaB2_210910_173733.vasp


Test all sqs's after mc sqs

In [12]:
with open('res.log') as f:
    data = f.readlines()
data

['Nat\tCS\tNa\tL\tCorr\tTarget\tDiff\n',
 '105\t4\t2\t3.180490\t0.352381\t0.360000\t-0.007619\n',
 '105\t4\t2\t3.545480\t0.314286\t0.360000\t-0.045714\n',
 '105\t5\t2\t3.180490\t0.352381\t0.360000\t-0.007619\n',
 '105\t5\t2\t3.545480\t0.314286\t0.360000\t-0.045714\n',
 '105\t5\t2\t4.762976\t0.352381\t0.360000\t-0.007619\n',
 '105\t6\t2\t3.180490\t0.352381\t0.360000\t-0.007619\n',
 '105\t6\t2\t3.545480\t0.314286\t0.360000\t-0.045714\n',
 '105\t6\t2\t4.762976\t0.352381\t0.360000\t-0.007619\n',
 '105\t6\t2\t5.508770\t0.352381\t0.360000\t-0.007619\n',
 '105\t7\t2\t3.180490\t0.352381\t0.360000\t-0.007619\n',
 '105\t7\t2\t3.545480\t0.314286\t0.360000\t-0.045714\n',
 '105\t7\t2\t4.762976\t0.352381\t0.360000\t-0.007619\n',
 '105\t7\t2\t5.508770\t0.352381\t0.360000\t-0.007619\n',
 '105\t7\t2\t6.360980\t0.352381\t0.360000\t-0.007619\n',
 '105\t7\t2\t6.551105\t0.352381\t0.360000\t-0.007619\n',
 '105\t8\t2\t3.180490\t0.352381\t0.360000\t-0.007619\n',
 '105\t8\t2\t3.545480\t0.314286\t0.360000\t-0.0